In [2]:
import time
import requests
from bs4 import BeautifulSoup
import pymongo
import pandas as pd
import numpy as np
import re

from sklearn.externals import joblib

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wnlzer = WordNetLemmatizer()


from pprint import pprint
from time import time
import logging


from collections import defaultdict
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity

import sys
import os
import csv
import shutil
from collections import defaultdict

from sklearn.linear_model import SGDClassifier, Perceptron
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score
#import parfit.parfit as pf

import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display

In [3]:

widgets.IntSlider(10)

IntSlider(value=10)

In [4]:
#Import the dataframe with strain review info

test_reviews = pd.read_csv("./../../new_full_test_strains_reviews.csv")

#Drop Spam users

new_tr = test_reviews.drop([3570,2728,355], axis=0)

new_tr.columns

Index(['Unnamed: 0', 'hybrid/blue-dream', 'sativa/sour-diesel', 'hybrid/gsc',
       'sativa/green-crack', 'hybrid/og-kush', 'indica/granddaddy-purple',
       'hybrid/original-glue', 'hybrid/white-widow', 'hybrid/blue-dream.1',
       'sativa/sour-diesel.1', 'hybrid/gsc.1', 'sativa/green-crack.1',
       'hybrid/og-kush.1', 'indica/granddaddy-purple.1',
       'hybrid/original-glue.1', 'hybrid/white-widow.1', 'sativa/jack-herer',
       'indica/bubba-kush', 'hybrid/pineapple-express', 'hybrid/trainwreck',
       'hybrid/ak-47', 'sativa/durban-poison', 'indica/northern-lights',
       'hybrid/headband', 'indica/blue-cheese', 'sativa/strawberry-cough',
       'hybrid/chemdawg', 'indica/purple-kush', 'sativa/lemon-haze',
       'sativa/super-lemon-haze', 'indica/grape-ape', 'indica/blueberry',
       'sativa/alaskan-thunder-fuck', 'sativa/super-silver-haze',
       'indica/blackberry-kush', 'hybrid/cherry-pie', 'indica/master-kush',
       'hybrid/skywalker-og', 'hybrid/cheese', 'indica/

In [5]:
#Look for spam strings in the df[columns](i.e. in a review sring)...return user or strain etc that has it. 

mask = np.column_stack([new_tr[col].str.contains(r"ke thick robust problem going coughing thats alway", na=False) for col in new_tr])
new_tr.loc[mask.any(axis=1)]

,Unnamed: 0,hybrid/blue-dream,sativa/sour-diesel,hybrid/gsc,sativa/green-crack,hybrid/og-kush,indica/granddaddy-purple,hybrid/original-glue,hybrid/white-widow,hybrid/blue-dream.1,...,indica/gods-gift,indica/la-confidential,hybrid/agent-orange,indica/purple-urkle,hybrid/lemon-kush,hybrid/mazar-x-blueberry,hybrid/golden-goat,indica/afghan-kush,hybrid/dutch-treat,indica/hindu-kush


In [6]:
#Drop dupplicate colums and user index

to_drop = ['Unnamed: 0','hybrid/blue-dream.1',
       'sativa/sour-diesel.1', 'hybrid/gsc.1', 'sativa/green-crack.1',
       'hybrid/og-kush.1', 'indica/granddaddy-purple.1',
       'hybrid/original-glue.1', 'hybrid/white-widow.1','hybrid/skywalker-og']

tr = new_tr.drop(to_drop, axis=1) 

#Save Strains as index for later...

final_strain_index = tr.columns
final_strain_index

Index(['hybrid/blue-dream', 'sativa/sour-diesel', 'hybrid/gsc',
       'sativa/green-crack', 'hybrid/og-kush', 'indica/granddaddy-purple',
       'hybrid/original-glue', 'hybrid/white-widow', 'sativa/jack-herer',
       'indica/bubba-kush', 'hybrid/pineapple-express', 'hybrid/trainwreck',
       'hybrid/ak-47', 'sativa/durban-poison', 'indica/northern-lights',
       'hybrid/headband', 'indica/blue-cheese', 'sativa/strawberry-cough',
       'hybrid/chemdawg', 'indica/purple-kush', 'sativa/lemon-haze',
       'sativa/super-lemon-haze', 'indica/grape-ape', 'indica/blueberry',
       'sativa/alaskan-thunder-fuck', 'sativa/super-silver-haze',
       'indica/blackberry-kush', 'hybrid/cherry-pie', 'indica/master-kush',
       'hybrid/cheese', 'indica/death-star', 'sativa/chocolope',
       'sativa/amnesia-haze', 'hybrid/tahoe-og', 'sativa/maui-wowie',
       'hybrid/platinum-gsc', 'sativa/harlequin', 'indica/gods-gift',
       'indica/la-confidential', 'hybrid/agent-orange', 'indica/purple-u

In [7]:
strain_lookup = {}
for strain, cnt in zip(list(final_strain_index),range(len(list(final_strain_index)))):
    strain_lookup[cnt] = strain

In [8]:
# Convert list of strain reviews strings into a single string for each strain;
# store strain string in list of all strain strings.

def cols_2_string(df):
    strain_list = []
    cols = df.columns
    for i in cols:
        strain = df[i].dropna().tolist()
        strain_str = " ".join(strain)
        strain_list.append(strain_str)
    return strain_list

col_string = cols_2_string(tr)
col_string

['purchased sqdc quebec blue dream aurora thc bought thought would strong sensitive thc loaded utilian g blue dream manit great felt mild head high came almost immediatly felt calm happy slightly uplifted energetic like pure sativa minute felt even relaxed throughout ehlers danlos syndrome constantly battling chronic pain blue dream one best come across relieve pain day time use also fight number stemming ailment ed tried first time today took two small hit vape waited minute thought hadnt vaped enough kicked high relaxing still get good head buzz well great chilling couch also productive want took almost pain away kicked happy blue dream dream right try smoking bed youll good night rest classic og kush blue dream forever brighten blue mood moon got relaxed put chair set haze watching tvenjoyed review vape im smoking actually cartridge abxi really looking forward trying fact maybe done thing read rave review couldnt wait get well tell feel pretty darn good right ive taken one hit minut

In [9]:
#Split each review it by white space, for each strain

texts = [[word for word in document.split()]for document in col_string]

#Count word frequencies for each strain

from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

#frequency

In [10]:
# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]

print(processed_corpus)

[['purchased', 'sqdc', 'quebec', 'blue', 'dream', 'aurora', 'thc', 'bought', 'thought', 'would', 'strong', 'sensitive', 'thc', 'loaded', 'g', 'blue', 'dream', 'great', 'felt', 'mild', 'head', 'high', 'came', 'almost', 'immediatly', 'felt', 'calm', 'happy', 'slightly', 'uplifted', 'energetic', 'like', 'pure', 'sativa', 'minute', 'felt', 'even', 'relaxed', 'throughout', 'syndrome', 'constantly', 'battling', 'chronic', 'pain', 'blue', 'dream', 'one', 'best', 'come', 'across', 'relieve', 'pain', 'day', 'time', 'use', 'also', 'fight', 'number', 'ailment', 'ed', 'tried', 'first', 'time', 'today', 'took', 'two', 'small', 'hit', 'vape', 'waited', 'minute', 'thought', 'hadnt', 'vaped', 'enough', 'kicked', 'high', 'relaxing', 'still', 'get', 'good', 'head', 'buzz', 'well', 'great', 'chilling', 'couch', 'also', 'productive', 'want', 'took', 'almost', 'pain', 'away', 'kicked', 'happy', 'blue', 'dream', 'dream', 'right', 'try', 'smoking', 'bed', 'youll', 'good', 'night', 'rest', 'classic', 'og', 'k

In [14]:
#Create dictionary from corpus words

#dictionary = corpora.Dictionary(processed_corpus)
#pprint(dictionary)


# store the dictionary, for future reference

#dictionary.save('new_strains.dict')  
tdict = corpora.Dictionary.load('new_strains.dict')
tdict

In [55]:
#Print the distinct words in the corus and their corresponding id

print(len(tdict.token2id))

5822


In [16]:
# dob2bow counts the number of occurrences of each distinct word, 
# converts the word to its integer word id and returns the result as a sparse vector
# returns the sparse vector for each strain in a list indexed by strains

corpus = [tdict.doc2bow(text) for text in texts]
corpus

len(corpus)

47

In [53]:
#Initiate TF-IDF model
#calculate tf-idf weights and matrix for strain lis 
tfidf = models.TfidfModel(corpus)
tfidf

In [18]:
#Transform the corpus list, fit it 

corpus_tfidf = tfidf[corpus]
corpus_tfidf

for doc in corpus_tfidf:
    print(doc)

[(0, 0.012541471560123455), (1, 0.008223751826152972), (2, 0.0054707820432803355), (3, 0.0032516676965347337), (4, 0.016764720574627922), (5, 0.01160225298588104), (6, 0.009429082340837325), (7, 0.015011913864816148), (8, 0.019634246666735525), (9, 0.019235162879320613), (10, 0.012541471560123455), (11, 0.00631150991374421), (12, 0.007830533500952527), (13, 0.008618717601889458), (14, 0.018036422432240292), (15, 0.02157732970062336), (16, 0.007587224625247712), (17, 0.013652331355341794), (18, 0.00769250163371701), (19, 0.012541471560123455), (20, 0.005518356208988048), (21, 0.016764720574627922), (22, 0.015011913864816148), (23, 0.01078866485031168), (24, 0.013652331355341794), (25, 0.005518356208988048), (26, 0.012284920593343563), (27, 0.010071029255430766), (28, 0.01078866485031168), (29, 0.007606821624966131), (30, 0.01078866485031168), (31, 0.012541471560123455), (32, 0.02014205851086153), (33, 0.02508294312024691), (34, 0.01078866485031168), (35, 0.019235162879320613), (36, 0.01

In [22]:
# Initiate the LSI (Latent Semnatic Indexing) model with dictionary that i created earlier 
# and transformed fit corpus

model = models.LsiModel(corpus_tfidf, id2word=tdict)

#model.save("fitted_LSI_Model.model")

In [51]:
lsi_fitted = model.load("fitted_LSI_Model.model")
lsi_fitted

In [24]:
#User input part...
user_input = "netflix and chill with girlfriend"

#Create BOW and LSI vector for the user input to

vec_bow = tdict.doc2bow(user_input.lower().split())
vec_lsi = model[vec_bow]
#print(vec_lsi)

In [28]:
# Create (nxn) strain similarity matrix to index from corpus
# transform corpus to LSI space 


#index = MatrixSimilarity(model[corpus]) 
#index
#index.save('strain_sim.index')
s_index = MatrixSimilarity.load('strain_sim.index')

In [29]:
#Index into the similarity matrix with the input vector
sims = s_index[vec_lsi]

#print(list(enumerate(sims)))

#sort the similarity list
sims_sort = sorted(enumerate(sims), key=lambda item: -item[1])

#return the top 5 strains and their score
top_5 = sims_sort[:5]
top_list =[]
for i in top_5:
    top_list.append((strain_lookup[i[0]],i[1]))
#print(top_list)

In [30]:
strain_lookup = {}
for strain, cnt in zip(list(final_strain_index),range(len(list(final_strain_index)))):
    strain_lookup[cnt] = strain

In [31]:
def input_to_vec(user_input,i):
    tdict = corpora.Dictionary.load('new_strains.dict')
    lsi_fitted = model.load("fitted_LSI_Model.model")
    index = MatrixSimilarity.load('strain_sim.index')
    vec_bow = tdict.doc2bow(user_input.lower().split())
    vec_lsi = model[vec_bow]
    sims = index[vec_lsi]
    sims_sort = sorted(enumerate(sims), key=lambda item: -item[1])
    top_5 = sims_sort[:i]
    top_list =[]
    for i in top_5:
        top_list.append((strain_lookup[i[0]],i[1]))
    return top_list

In [32]:
input_to_vec('ski on mountain',6)

[('indica/master-kush', 0.5258174),
 ('indica/hindu-kush', 0.29244345),
 ('hybrid/cheese', 0.26313818),
 ('indica/bubba-kush', 0.14175251),
 ('hybrid/og-kush', 0.059499294),
 ('sativa/strawberry-cough', 0.032069918)]

In [33]:
t_box = widgets.Text(
    placeholder='Type something',
    description='Make me:',
    disabled=False
)

In [34]:
slide = widgets.IntSlider(
    min=2,
    max=10,
    step=1,
    description='# Strains:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [35]:
button = widgets.Button(
    description="Let's get Lit!",
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='leaf'
)

In [36]:

def st_2_vec(a,b):
        tdict = corpora.Dictionary.load('new_strains.dict')
        lsi_fitted = model.load("fitted_LSI_Model.model")
        index = MatrixSimilarity.load('strain_sim.index')

        vec_bow = tdict.doc2bow(str(a).lower().split())
        vec_lsi = model[vec_bow]
        sims = index[vec_lsi]
        sims_sort = sorted(enumerate(sims), key=lambda item: -item[1])
        top_5 = sims_sort[:]
        top_list =[]
        for i in top_5:
            top_list.append((strain_lookup[i[0]],i[1]))
        return top_list

In [49]:
ui = widgets.VBox([t_box, slide, button])
results = widgets.Output()
 
@results.capture(clear_output=True)
def st_2_vec():
        x = slide.value
        y = t_box.value
        
        tdict = corpora.Dictionary.load('new_strains.dict')
        lsi_fitted = model.load("fitted_LSI_Model.model")
        index = MatrixSimilarity.load('strain_sim.index')

        vec_bow = tdict.doc2bow(str(y).lower().split())
        vec_lsi = model[vec_bow]
        sims = index[vec_lsi]
        sims_sort = sorted(enumerate(sims), key=lambda item: -item[1])
        top_5 = sims_sort[:x]
        tops_list =[]
        for i in top_5:
            tops_list.append((strain_lookup[i[0]],i[1]))
        print("Give these a try {}".format(tops_list))
    
@button.on_click
def _click(b):
    st_2_vec(x,y)
        
button.on_click(_click)
display(ui, results)

Output()